In [18]:
!pip install -q keras jupyternotify

%load_ext jupyternotify
import keras

import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Activation, Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.utils import np_utils

from keras.datasets import mnist

import matplotlib.pyplot as plt

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [19]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print(train_images.shape)
print(train_labels.shape)

(60000, 28, 28)
(60000,)


In [20]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

test_labels = np_utils.to_categorical(test_labels, 10)
train_labels = np_utils.to_categorical(train_labels, 10)

print(train_images.shape)
print(train_labels.shape)

(60000, 28, 28, 1)
(60000, 10)


In [110]:
%%notify

model = Sequential()
dropout = 0.05

def add_conv2d( ksize, **kwargs):
    model.add(Conv2D(ksize, 3, activation='relu', use_bias=False, **kwargs))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))

add_conv2d(24, input_shape=train_images.shape[1:])
add_conv2d(16)
add_conv2d(16)
model.add(MaxPooling2D(2))
add_conv2d(16)
add_conv2d(16)
add_conv2d(10)
add_conv2d(10)
add_conv2d(10)

model.add(Flatten())
model.add(Activation('softmax'))

model.summary()


model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=3e-3), 
    # optimizer='adam',
    metrics=['accuracy']
)


def scheduler(epoch, lr):
    return 3e-3 / (1 + 0.5 * epoch)

model.fit(
    train_images, train_labels, 
    epochs=20,
    validation_data=[test_images, test_labels],
    verbose=1,
    batch_size=256,
    callbacks=[LearningRateScheduler(scheduler, verbose=1)],
)

Model: "sequential_90"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_575 (Conv2D)          (None, 26, 26, 24)        216       
_________________________________________________________________
batch_normalization_574 (Bat (None, 26, 26, 24)        96        
_________________________________________________________________
dropout_521 (Dropout)        (None, 26, 26, 24)        0         
_________________________________________________________________
conv2d_576 (Conv2D)          (None, 24, 24, 16)        3456      
_________________________________________________________________
batch_normalization_575 (Bat (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_522 (Dropout)        (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_577 (Conv2D)          (None, 22, 22, 16)      

<IPython.core.display.Javascript object>

In [111]:
model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 2s 245us/step


[0.023790680991858243, 0.9939]